# <b>(Phase 2 | Search 1) Abstract Screening & MAXQDA Preparation: Combining ERIC API with CrossRef to Develop RIS File & Download PDFs</b>
---
This Jupyter Notebook provides an analysis memo detailing the steps taken for the systematic literature review, including manual screening of abstracts, data cleaning, integration with the CrossRef API, RIS file creation, and PDF collection.

<b>Project:</b> Systematic Literature Review of Tracking in P-20 Education<br>

<b> Notebook Overview:</b>

This notebook comprises the following key sections:

<b>1. Abstract Screening and Data Cleaning:</b>
<ul> <li>Manual screening of abstracts to include only relevant articles.</li> <li>Data cleaning and filtering of the ERIC database to retain included records.</li> </ul>

<b>2. CrossRef API Integration:</b>
<ul> <li>Connects to the CrossRef API to enrich the ERIC records with additional metadata such as abstracts, authors, and publication details.</li> <li>Ensures comprehensive data retrieval with custom handling for specific content types and publisher links.</li> </ul>

<b>3. RIS File Development:</b>
<ul> <li>Maps DataFrame columns to RIS tags to prepare the data for export as an RIS file.</li> <li>Processes fields like authors, publication type, and abstract while handling various bibliographic data formats.</li> </ul>

<b>4. PDF Downloading:</b>
<ul> <li>Automates the process of downloading PDFs from Sci-Hub using DOIs.</li> <li>Organizes downloaded PDFs into decade-specific subfolders and formats filenames based on author and title.</li> </ul>

---
### <ins>Analysis Memo: Abstract Screening & DOI Cross-reference</ins>

This section of the Jupyter notebook reviews the abstract screening for Search 1 articles. The team examined the ERIC information (abstracts, titles, thesaurus terms, journals, etc.) to determine whether each article should be included or excluded.

#### Search 1: Abstract Screening
| Row Labels                          | Count of id |
|--------------------------------------|-------------|
| Include                              | 644         |
| Exclude                              | 84          |
| Exclude (Article Not Relevant)       | 456         |
| Exclude (Literature Review)          | 1           |
| Exclude (Non-US Article Context)     | 4           |
| Exclude (Reference)                  | 16          |
| Exclude (Validation Database)        | 166         |
| Exclude                              | 2           |
| **Grand Total**                      | **1,373**   |

Due to the large number of articles, we sought a method to import the database into Endnote in bulk and collect PDFs. Our research indicated that DOIs are the most reliable way to capture this information. However, since ERIC has not retroactively updated the URL field to include DOIs added by publishers after publication, we aimed to find a way to capture more DOIs. We developed several Excel formulas (e.g., DOI Formula and URL Found) to identify URLs with DOIs and standardize them (Excel file: Phase 2D - Search 1 Checks). With this information, we wrote a Python script to import the Phase 2D - Abstract Screening file, normalize the titles (to lowercase), and use exact and fuzzy matching to find reference information (title, DOI, URL, citation count) from the CrossRef API using the Habanero client.

#### Search 1: CrossRef Matching Analysis
After using Crossref matching, there was a substantial increase in the number of DOIs found. Initially, only 200 DOIs were identified (31%), but this number increased to 533 (83%) after matching. Additionally, records without a URL decreased significantly, from 61% to 14%

##### <b>Search 1: Pre-Cross Reference Matching</b>
| Row Labels   | Count of id | Percentage |
|--------------|-------------|------------|
| DOI Found    | 200         | 31%        |
| No URL       | 391         | 61%        |
| Other URL    | 53          | 8%         |
| **Grand Total** | **644**  | **100%**   |

##### <b>Search 1: Pre-Cross Reference Matching</b>
| Row Labels   | Count of id | Percentage |
|--------------|-------------|------------|
| DOI Found    | 533         | 83%        |
| No URL       | 91          | 14%        |
| Other URL    | 20          | 3%         |
| **Grand Total** | **644**  | **100%**   |

We created a CSV version of the CrossRef file and used an XLOOKUP function to join the information with Phase 2d - Abstract Screening. Using the collected data, we developed a "Continued URL" column and a "DOI_FORMULA_POST2" column to capture the DOI URLs across both the ERIC and CrossRef columns, creating the relevant categories in the table. For the 91 articles without a URL, we recorded them in the "Missing URLs" column. Several articles had incorrect titles from ERIC. Approximately 15 articles had full texts available on ERIC.ED.GOV, while others had direct PDFs from journals, and a large portion had a DOI, JSTOR stable link, or PMID. Articles that could not be located were requested as PDFs through the UGA interlibrary loan program.

##### <b>Search 1: Post CF & Missing Search</b>

| Row Labels                     | Count of id | Percentage |
|---------------------------------|-------------|------------|
| DOI Found                      | 551         | 86%        |
| Other URL                      | 64          | 10%        |
| No URL: Requested from Library | 29          | 5%         |
| **Grand Total**                | **644**     | **100%**   |

After using addressing missing url/dois with google searches, there was a noticeable improvement in the number of DOIs found. 533 DOIs were identified (83%), which increased to 551 (86%) post-matching. Additionally, the number of records without a URL decreased, from 91 (14%) to just 29 (5%), reflecting an overall enhancement in data completeness after further processing.


The code below uses the Phase 2d - abstract screening file and the URLS to identify articles that need additional searching vs those that were found with a doi or URL, create RIS output for endnote that includes the ERIC ID, DOI, and other relevant items to create a , and  



---
## Import the modified "Search 1" excel file that the research team screened the abstracts for

In [25]:
import pandas as pd
from IPython.display import display

# URL of the Screening2 Excel file from your GitHub repository
file_url = 'https://raw.githubusercontent.com/s-baser/P20Tracking_Review/main/ManualScreeningFiles/Screening2/phase2d_search1_NoFormula.xlsx'

# Load the entire sheet from the Excel file and display the first few rows
df_full_sheet = pd.read_excel(file_url, sheet_name=0)

# Slice the DataFrame to extract the relevant table (A1 to AP1374)
Abstract_Screening_1 = df_full_sheet.iloc[0:1374, 0:42]  # Adjusted to cover rows 1-1374 and columns A-AP

# Filter out records that are not "Search 1" in the 'source_search' column
Abstract_Screening_1 = Abstract_Screening_1[Abstract_Screening_1['source_search'] == 'Search 1']

# Display total records loaded after filtering
print("Initial Data Load after filtering for 'Search 1'")
print(f"Total records loaded: {Abstract_Screening_1.shape[0]}")

# Print the total number of articles by 'source_search'
print("\nTotal number of articles by 'source_search':")
print(Abstract_Screening_1['source_search'].value_counts())

# Set display options for scrollable view if the DataFrame is large
pd.options.display.max_rows = 5  # Adjust as needed for number of rows to display at once
pd.options.display.max_columns = 10  # Adjust as needed for number of columns to display at once

# Display the DataFrame
display(Abstract_Screening_1)

Initial Data Load after filtering for 'Search 1'
Total records loaded: 1373

Total number of articles by 'source_search':
source_search
Search 1    1373
Name: count, dtype: int64


,id,source_search,title,author,publicationdateyear,...,Researcher,Decision,Notes,Amy Feedback,Combined_URL_cf_missing
0,EJ199000,Search 1,Curriculum Tracking and Educational Stratifica...,"Alexander, Karl L., And Others",1978,...,Sean,Exclude(Validation Database),NaN,NaN,No URL
1,EJ198024,Search 1,Services for Learning Disabled Adolescents: A ...,"McNutt, Gaye, Heller, Ginger",1978,...,Sean,Exclude(Article Not Relevant),Doesn't seem like it is empirical based on web...,NaN,No URL
...,...,...,...,...,...,...,...,...,...,...,...
1371,EJ1426302,Search 1,Tracking the Effects: Examining the Opportunit...,"Kristian Edosomwan, Jemimah L. Young, Bettie R...",2024,...,Carlie,Include,NaN,NaN,https://doi.org/10.1177/00220574231168634
1372,EJ1418150,Search 1,The Work Integrated Learning Experience of a U...,"Fiona Rillotta, Lorraine Lindsay, Cassandra Gi...",2024,...,Carlie,Exclude,NaN,NaN,https://doi.org/10.1080/13603116.2021.1937343


------
## <b> Search 1: Preparing Articles for MAXQDA </b>

<ul>
    <li>Clean ERIC API & Manual Data</li>
    <li>DOI crossreference</li>
    <li>RIS Creation</li>
    <li>PDF download</li>

</ul>

### ERIC Data Edits/Cleaning

This notebook performs the following steps to clean and modify the dataset:

<b>1. Initial Data Preparation:</b>
<ul> <li>Load the data from the Excel sheet.</li> <li>Filter to include only rows where <code>Decision</code> is "Include."</li> </ul>

<b>2. Column Renaming and Prefixing:</b>
<ul> <li>Add an <code>ERIC_</code> prefix to the first 34 columns.</li> <li>Add a <code>d_</code> prefix to derived variables.</li> </ul>

<b>3. Column Modifications:</b>
<ul> <li>Rename <code>Combined_URL_cf_missing</code> to <code>d_URL_cf_missing</code>.</li> <li>Create <code>d_doi_cf_missing</code> by extracting the DOI from <code>d_URL_cf_missing</code>.</li> </ul>

<b>4. Data Extraction:</b>
<ul> <li>Extract volume, issue, pages, and date from <code>ERIC_sourceid</code> using regex.</li> <li>Adjust <code>ERIC_endpage</code> for consistent page formatting.</li> </ul>

<b>5. New Column Creation:</b>
<ul> <li>Create <code>ERIC_page_range</code> combining <code>ERIC_startpage</code> and <code>ERIC_endpage</code>.</li> <li>Convert <code>ERIC_source</code> to title case.</li> </ul>

<b>6. Column Removal:</b>
<ul> <li>Remove unnecessary columns: <code>source_screen_5</code>, <code>Validation Database</code>, <code>Researcher</code>, <code>Decision</code>, <code>Notes</code>, and <code>Amy Feedback</code>.</li> </ul>

<b>7. Column Reordering:</b>
<ul> <li>Reorder columns for logical grouping and presentation.</li> </ul>

In [31]:
import pandas as pd
import re

# ----------------------------
# Load the DataFrame
# ----------------------------

# Remove rows where Decision is not "Include"
phase2d_S1_df = Abstract_Screening_1[Abstract_Screening_1['Decision'] == "Include"].copy()


# Add "ERIC_" prefix to the first 34 columns
for i, col in enumerate(phase2d_S1_df.columns):
    if i <= 34:  # Modify only the first 34 columns
        phase2d_S1_df.rename(columns={col: f"ERIC_{col}"}, inplace=True)

# Rename "Combined_URL_cf_missing" (which is the 35th column) to "d_URL_cf_missing"
phase2d_S1_df.rename(columns={'Combined_URL_cf_missing': 'd_URL_cf_missing'}, inplace=True)

# List of columns to drop
columns_to_drop = ['source_screen_5', 'Validation Database', 'Researcher', 'Decision', 'Notes', 'Amy Feedback']

# Drop the specified columns
phase2d_S1_df.drop(columns=columns_to_drop, inplace=True)

# Create a derived variable (d_doi_cf_missing)
phase2d_S1_df['d_doi_cf_missing'] = phase2d_S1_df['d_URL_cf_missing'].apply(
    lambda x: x.split('doi.org/')[1] if isinstance(x, str) and 'doi.org/' in x else ''
)

# ----------------------------
# Extract Volume, Issue, Pages, and Date from ERIC_sourceid
# ----------------------------

# Regular expression pattern to extract volume, issue, pages, and date
pattern = r'v(?P<volume>\d+)\s+n(?P<issue>\d+)\s+p(?P<start_page>\d+)-(?P<end_page>\d+)\s+(?P<date>[\w\-]+)\s+\d{4}'

# Function to fix the end page
def fix_end_page(start_page, end_page):
    if start_page and end_page:
        start_page_str = str(start_page)
        end_page_str = str(end_page)
        # If end_page is shorter than start_page, prepend leading digits
        if len(end_page_str) < len(start_page_str):
            prefix_length = len(start_page_str) - len(end_page_str)
            fixed_end_page = start_page_str[:prefix_length] + end_page_str
            return fixed_end_page
    return end_page

# Function to extract the components using regex
def extract_source_details(source_id):
    match = re.search(pattern, source_id)
    if match:
        volume = match.group('volume')
        issue = match.group('issue')
        start_page = match.group('start_page')
        end_page = match.group('end_page')
        date = match.group('date')
        # Fix the end page if necessary
        end_page = fix_end_page(start_page, end_page)
        return volume, issue, start_page, end_page, date
    return None, None, None, None, None

# Apply the function to extract components into new columns
phase2d_S1_df[['ERIC_volume', 'ERIC_issue', 'ERIC_startpage', 'ERIC_endpage', 'ERIC_date']] = phase2d_S1_df['ERIC_sourceid'].apply(
    lambda x: pd.Series(extract_source_details(x))
)

# ----------------------------
# Create ERIC_page_range Column
# ----------------------------

# Function to create page range
def create_page_range(start_page, end_page):
    if start_page and end_page:
        return f"{start_page}-{end_page}"
    elif start_page:
        return start_page
    elif end_page:
        return end_page
    else:
        return ''

# Apply the function to create ERIC_page_range column
phase2d_S1_df['ERIC_page_range'] = phase2d_S1_df.apply(
    lambda row: create_page_range(row['ERIC_startpage'], row['ERIC_endpage']), axis=1
)

# ----------------------------
# Convert ERIC_source to Title Case
# ----------------------------

# Update the ERIC_source column to Title Case
phase2d_S1_df['ERIC_source'] = phase2d_S1_df['ERIC_source'].apply(
    lambda x: x.title() if isinstance(x, str) else x
)

# ----------------------------
# Reorder and Display DataFrame
# ----------------------------

# Define the correct order of columns
ordered_columns = [
    'ERIC_id', 'ERIC_source_search', 'ERIC_title', 'ERIC_author', 'ERIC_publicationdateyear', 'ERIC_description',
    'ERIC_subject', 'ERIC_source', 'ERIC_publicationtype', 'ERIC_peerreviewed', 'ERIC_e_fulltextauth', 'ERIC_sourceid',
    'ERIC_volume', 'ERIC_issue', 'ERIC_startpage', 'ERIC_endpage', 'ERIC_page_range', 'ERIC_date', 'ERIC_language', 'ERIC_issn',
    'ERIC_e_yearadded', 'ERIC_e_datemodified', 'ERIC_identifiersgeo', 'ERIC_authorxlink', 'ERIC_audience',
    'ERIC_identifierslaw', 'ERIC_publisher', 'ERIC_educationlevel', 'ERIC_url', 'ERIC_abstractor', 'ERIC_iescited',
    'ERIC_identifierstest', 'ERIC_sponsor', 'ERIC_iesgrantcontractnum', 'ERIC_iesgrantcontractnumxlink', 'ERIC_ieslinkdatasource',
    'ERIC_institution', 'ERIC_iesfunded', 'ERIC_ieswwcreviewed', 'ERIC_ieslinkwwcreviewguide', 'ERIC_isbn',
    'd_URL_cf_missing', 'd_doi_cf_missing'
]


# Print the total number of articles by 'source_search'
print("\nTotal number of articles by 'source_search':")
print(phase2d_S1_df['ERIC_source_search'].value_counts())

# Reorder the columns in the DataFrame
phase2d_S1_df = phase2d_S1_df[ordered_columns]

# Display the first few rows of the updated DataFrame
display(phase2d_S1_df.head(10))


Total number of articles by 'source_search':
ERIC_source_search
Search 1    641
Name: count, dtype: int64


,ERIC_id,ERIC_source_search,ERIC_title,ERIC_author,ERIC_publicationdateyear,...,ERIC_ieswwcreviewed,ERIC_ieslinkwwcreviewguide,ERIC_isbn,d_URL_cf_missing,d_doi_cf_missing
4,EJ199088,Search 1,Ability and Group Problem Solving.,"Laughlin, Patrick R.",1978,...,NaN,NaN,NaN,No URL,
5,EJ198017,Search 1,Mathematics and Learning Disabled Youth: The U...,"Cawley, John F., And Others",1978,...,NaN,NaN,NaN,https://doi.org/10.2307/1510976,10.2307/1510976
...,...,...,...,...,...,...,...,...,...,...,...
13,EJ211808,Search 1,The Cloze Procedure: Some New Applications.,"Legenza, Alice, Elijah, David",1979,...,NaN,NaN,NaN,https://doi.org/10.1080/00220671.1979.10885189,10.1080/00220671.1979.10885189
14,EJ203992,Search 1,Student Achievement on Different Types of Task...,"Johnson, David W., And Others",1979,...,NaN,NaN,NaN,https://doi.org/10.1016/0361-476x(79)90063-8,10.1016/0361-476x(79)90063-8


### Crossref Data Extraction and Merging

This notebook automates the process of querying Crossref for metadata and abstracts using DOIs from a dataset, processing the returned data, and exporting the results to an Excel file.
Steps in the Workflow:

<b>1. Initialize Crossref Client:</b>
<ul> <li>Sets up the Crossref client using the <code>habanero</code> library.</li> <li>Defines a function to query Crossref with a DOI and extract relevant metadata, including the abstract, author information, and publication details.</li> </ul>

<b>2. Process DataFrame Entries:</b>
<ul> <li>Filters the input DataFrame to retain only rows with non-empty DOIs.</li> <li>Iterates through DOIs, querying Crossref for metadata and formatting specific fields (e.g., <code>cf_title</code>, <code>cf_author</code>, <code>cf_link</code>).</li> <li>Handles specific cases for different publishers and content types (e.g., Elsevier, Wiley, SAGE) to construct or exclude links.</li> </ul>

<b>3. Combine and Clean Data:</b>
<ul> <li>Removes existing <code>cf_</code> columns from the original DataFrame to avoid duplication.</li> <li>Concatenates the original DataFrame with the newly extracted Crossref data.</li> </ul>

<b>4. Save Processed Data to Excel:</b>
<ul> <li>Ensures the output directory exists and saves the final DataFrame as an Excel file.</li> <li>Provides the path to the saved file for user reference.</li> </ul>

<b>Output</b>

<ul> 
    <li>The notebook outputs an Excel file named Search_1_ERIC-CF_Export.xlsx stored in the specified directory.</li> 
    <li>The file includes enriched metadata and abstract information for each DOI, formatted with cf_ prefixes to indicate Crossref-sourced data.</li></ul>

<b>Prerequisites</b>

Python Libraries: <ul> <li><code>pandas</code>: DataFrame manipulation</li> <li><code>habanero</code>: Crossref API client</li> <li><code>tqdm</code>: Progress bar for loops</li> <li><code>BeautifulSoup4</code>: HTML/XML parsing</li> <li><code>os</code>: File path handling</li> <li><code>re</code>: Regular expressions for text processing</li> </ul>
    Environment Setup: <ul> <li>Ensure Python is installed, and the notebook is run in an environment (e.g., Jupyter Notebook, JupyterLab) that supports these libraries.</li> <li>Access to the Crossref API, with the `mailto` parameter set to your email for compliant queries.</li> </ul>

In [40]:
import pandas as pd
from habanero import Crossref
from tqdm import tqdm  # For progress bar
from bs4 import BeautifulSoup  # Ensure BeautifulSoup is installed
import os  # For handling file paths
import re  # For regular expressions

# ------------------------------
# 1. Initialize Crossref Client
# ------------------------------
cr = Crossref(mailto='sean.baser@uga.edu')

# Function to query Crossref and get the required fields
def query_crossref(doi):
    try:
        # Query Crossref with the DOI
        result = cr.works(ids=doi)
        
        # Extract abstract and handle JATS tags
        abstract = result['message'].get('abstract', '')
        if abstract:
            # Parse the abstract with BeautifulSoup
            soup = BeautifulSoup(abstract, 'lxml')
            # Extract text content
            abstract_text = soup.get_text()
        else:
            abstract_text = ''
        
        # Extract links and process according to publisher-specific rules
        links = result['message'].get('link', [])
        processed_links = []
        for link in links:
            url = link.get('URL', '')
            content_type = link.get('content-type', '')
            # Exclude links with 'text/xml' content-type
            if content_type == 'text/xml':
                continue
            # Check for Elsevier API link
            if 'api.elsevier.com/content/article/PII:' in url:
                # Extract the PII code using regex
                match = re.search(r'PII:([^?]+)', url)
                if match:
                    pii_code = match.group(1)
                    # Construct the new URL
                    new_url = f'https://www.sciencedirect.com/science/article/abs/pii/{pii_code}'
                    if new_url not in processed_links:
                        processed_links.append(new_url)
            # Check for Wiley API link
            elif 'api.wiley.com/onlinelibrary/tdm/v1/articles/' in url:
                # Exclude the Wiley API link and keep the direct PDF link
                continue  # Skip the Wiley API link
            # Exclude SAGE XML links
            elif 'journals.sagepub.com/doi/full-xml/' in url:
                continue  # Skip SAGE XML link
            # Exclude Emerald XML links
            elif '/full/xml' in url:
                continue  # Skip Emerald XML link
            # For Springer, keep only the first PDF link
            elif 'link.springer.com' in url:
                if url.endswith('.pdf'):
                    # Only include the first PDF link
                    if not any('link.springer.com' in l for l in processed_links):
                        processed_links.append(url)
            else:
                if url not in processed_links:
                    processed_links.append(url)
        
        # Make sure only one link is included
        if processed_links:
            cf_link = processed_links[0]  # Take the first link
        else:
            cf_link = ''
        
        # Extract authors, format as 'Surname, Given Name', separated by '; '
        cf_author = '; '.join([f"{author.get('family', '')}, {author.get('given', '')}" for author in result['message'].get('author', [])])
        
        # Extract contributors from various roles, format as 'Surname, Given Name', separated by '; '
        contributor_types = ['author', 'editor', 'chair', 'translator', 'contributor']  # List of possible contributor types
        contributors_list = []
        for ctype in contributor_types:
            for person in result['message'].get(ctype, []):
                family = person.get('family', '')
                given = person.get('given', '')
                name = f"{family}, {given}" if family or given else ''
                if name:
                    contributors_list.append(name)
        # Remove duplicates and join contributors
        cf_contributor = '; '.join(sorted(set(contributors_list), key=contributors_list.index))
        
        # Extract relevant fields with cf_ prefix
        cf_data = {
            'cf_title': result['message'].get('title', [''])[0],
            'cf_author': cf_author,
            'cf_DOI': result['message'].get('DOI', ''),
            'cf_publisher': result['message'].get('publisher', ''),
            'cf_journal_name': result['message'].get('container-title', [''])[0],
            'cf_ISSN': ', '.join(result['message'].get('ISSN', [])),
            'cf_ISBN': ', '.join(result['message'].get('ISBN', [])),
            'cf_issue': result['message'].get('issue', ''),
            'cf_volume': result['message'].get('volume', ''),
            'cf_page_range': result['message'].get('page', ''),
            'cf_published_print': result['message'].get('published-print', {}).get('date-parts', [['']])[0][0],
            'cf_published_online': result['message'].get('published-online', {}).get('date-parts', [['']])[0][0],
            'cf_citation_count': result['message'].get('is-referenced-by-count', 0),
            'cf_reference_count': result['message'].get('reference-count', 0),
            'cf_subject': ', '.join(result['message'].get('subject', [])),
            'cf_license': ', '.join([license['URL'] for license in result['message'].get('license', [])]),
            'cf_URL': result['message'].get('URL', ''),
            'cf_affiliation': ', '.join([affil.get('name', '') for affil in result['message'].get('author', [{}])[0].get('affiliation', [])]),
            'cf_funders': ', '.join([funder.get('name', '') for funder in result['message'].get('funder', [])]),
            'cf_type': result['message'].get('type', ''),
            'cf_language': result['message'].get('language', ''),
            'cf_abstract': abstract_text,
            'cf_publisher_location': result['message'].get('publisher-location', ''),
            'cf_contributor': cf_contributor,
            'cf_ORCID': ', '.join([author.get('ORCID', '') for author in result['message'].get('author', []) if 'ORCID' in author]),
            'cf_link': cf_link,
            'cf_deposited': result['message'].get('deposited', {}).get('date-parts', [['']])[0][0],
            'cf_score': result['message'].get('score', ''),
            'cf_license_URL': ', '.join([license.get('URL', '') for license in result['message'].get('license', [])]),
            'cf_event': result['message'].get('event', {}).get('name', ''),
            'cf_part_number': result['message'].get('part-number', '')
        }
        
        return cf_data
    
    except Exception as e:
        # Return empty dictionary if the query fails
        return {f'cf_{field}': '' for field in [
            'title', 'author', 'DOI', 'publisher', 'journal_name', 'ISSN', 'ISBN', 'issue', 'volume', 'page_range',
            'published_print', 'published_online', 'citation_count', 'reference_count', 'subject', 'license', 'URL',
            'affiliation', 'funders', 'type', 'language', 'abstract', 'publisher_location', 'contributor', 'ORCID', 
            'link', 'deposited', 'score', 'license_URL', 'event', 'part_number'
        ]}


# ------------------------------
# 2. Process All Entries with Non-Empty DOIs
# ------------------------------
# Assuming 'phase2d_S1_df' is your DataFrame from previous steps
# Filter rows with non-empty DOIs
phase2d_S1_df = phase2d_S1_df[phase2d_S1_df['d_doi_cf_missing'].notna()]

# Initialize a list to store the Crossref data for each row
crossref_data_list = []

# Use tqdm for the progress bar and iterate through all DOIs in the dataset
for doi in tqdm(phase2d_S1_df['d_doi_cf_missing']):
    crossref_data_list.append(query_crossref(doi))

# Convert the list of dictionaries into a DataFrame
crossref_df = pd.DataFrame(crossref_data_list)

# --- Remove existing cf_ columns to avoid duplication ---
# Identify columns that start with 'cf_'
cf_columns = [col for col in phase2d_S1_df.columns if col.startswith('cf_')]
# Drop these columns from the original DataFrame
phase2d_S1_df = phase2d_S1_df.drop(columns=cf_columns)

# Concatenate the original DataFrame with the Crossref data to create a new dataframe
phase2d_S1_cf_df = pd.concat([phase2d_S1_df.reset_index(drop=True), crossref_df], axis=1)

# Set Pandas options to display more content in cells
pd.set_option('display.max_colwidth', None)

# ------------------------------
# 3. Save Excel Output to Specified Folder
# ------------------------------

# Define the output directory and filename
output_dir = r"C:\Users\<USER>\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Phase 2 - Literature Review Search\ERIC API Resources\Search_1_ERIC-CF_Exports"
output_filename = "Search_1_ERIC-CF_Export.xlsx"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Combine directory and filename to create the full output path
output_excel_path = os.path.join(output_dir, output_filename)

# Write directly to an Excel file
phase2d_S1_cf_df.to_excel(output_excel_path, index=False)

print(f"Data successfully written to {output_excel_path}")

100%|██████████| 641/641 [11:59<00:00,  1.12s/it]


Data successfully written to C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Phase 2 - Literature Review Search\ERIC API Resources\Search_1_ERIC-CF_Exports\Search_1_ERIC-CF_Export.xlsx


### Generating and Exporting RIS Data from ERIC Dataset

This notebook processes a DataFrame from a structured ERIC dataset, maps relevant data to RIS fields, and exports the data into Excel and RIS formats. The RIS format is commonly used for managing bibliographic data in reference management software.
Steps in the Workflow:

<b>1. Prepare the DataFrame:</b>
<ul> <li>Replace <code>NaN</code> values with empty strings to avoid issues during data mapping.</li> </ul>

<b>2. Define Helper Functions:</b>
<ul> <li>Include functions to handle data extraction and formatting for RIS fields.</li> <li>Process the <code>ERIC_author</code> field to format author names as "Surname, Given Name."</li> </ul>

<b>3. Map RIS Fields to DataFrame Columns:</b>
<ul> <li>Map DataFrame columns to RIS fields using a dictionary for direct and custom mappings.</li> <li>Apply special processing for columns like <code>RIS_A1</code> (authors) and <code>RIS_UR</code> (URLs) to handle complex cases.</li> </ul>

<b>4. Export the DataFrame to Excel:</b>
<ul> <li>Save the modified DataFrame as an Excel file for review and further analysis.</li> </ul>

<b>5. Display the First Few Rows (Optional):</b>
<ul> <li>Display columns that start with <code>RIS_</code> for a quick preview of the RIS data structure.</li> </ul>

<b>6. Convert RIS Columns to RIS File:</b>
<ul> <li>Map columns to RIS tags and create entries following the RIS format specifications.</li> <li>Write the generated RIS content to a file, ensuring each record is formatted according to bibliographic standards.</li> </ul>

<b>Output</b>
<ul>
    <li>Excel File: The notebook exports the data as an Excel file named Search_1_ERIC-CF_RIS_Export.xlsx in the specified directory.</li>
    <li>RIS File: The notebook creates an RIS file named Search_1_ERIC-CF_RIS_Export.ris, suitable for import into reference management tools.</li>
</ul>

<b> Prerequisites</b>

Python Libraries: <ul> <li><code>pandas</code>: DataFrame manipulation</li> <li><code>numpy</code>: Handling NaN values</li> <li><code>os</code>: File path handling</li> <li><code>re</code>: Regular expressions for text processing</li> </ul>
Environment Setup: <ul> <li>Ensure Python is installed and the notebook runs in an environment (e.g., Jupyter Notebook, JupyterLab) that supports these libraries.</li> </ul>

In [51]:
import pandas as pd
import numpy as np
import os
import re  # Import re for regular expressions

# ------------------------------
# 1. Prepare the DataFrame
# ------------------------------

# Assuming 'phase2d_S1_cf_df' is your DataFrame from previous steps
# Replace any NaNs with empty strings to avoid issues during mapping
phase2d_S1_cf_ris_df = phase2d_S1_cf_df.replace({np.nan: ''})

# ------------------------------
# 2. Define Helper Functions
# ------------------------------

def get_ris_value(row, primary_col, secondary_cols=None, default=""):
    """
    Retrieves the value from the primary column if available; otherwise, checks secondary columns.
    """
    value = row.get(primary_col, '')
    if not value and secondary_cols:
        for sec_col in secondary_cols:
            value = row.get(sec_col, '')
            if value:
                break
    if not value:
        value = default
    return value

def process_eric_author(eric_author):
    """
    Processes the ERIC_author field to extract author names in 'Surname, Given Name' format.
    """
    # Remove 'And Others' and any preceding comma
    eric_author = eric_author.strip()
    eric_author = re.sub(r',?\s*And Others$', '', eric_author, flags=re.IGNORECASE)
    # Split by commas
    parts = [part.strip() for part in eric_author.split(',') if part.strip()]
    # Group names into pairs (LastName, FirstName MiddleName)
    authors = []
    i = 0
    while i < len(parts) - 1:
        last_name = parts[i]
        first_name = parts[i+1]
        authors.append(f"{last_name}, {first_name}")
        i += 2
    # Handle odd number of parts
    if i < len(parts):
        last_name = parts[i]
        authors.append(last_name)
    return authors

# ------------------------------
# 3. Map RIS Fields to DataFrame Columns
# ------------------------------

# Update the mapping for RIS_A1 to include ERIC_author as a secondary source
# Remove 'RIS_L2' from the mapping to prevent duplication
ris_fields = {
    'RIS_AN': ('ERIC_id',),
    'RIS_TY': ('cf_type', 'ERIC_publicationtype'),
    'RIS_T1': ('cf_title', 'ERIC_title'),
    'RIS_A1': ('cf_author', 'ERIC_author'),
    'RIS_DO': ('d_doi_cf_missing', 'cf_DOI'),
    'RIS_PB': ('cf_publisher', 'ERIC_publisher'),
    'RIS_JO': ('cf_journal_name', 'ERIC_source'),
    'RIS_SN': ('cf_ISSN', 'ERIC_issn'),
    'RIS_ISBN': ('cf_ISBN', 'ERIC_isbn'),
    'RIS_IS': ('cf_issue', 'ERIC_issue'),
    'RIS_VL': ('cf_volume', 'ERIC_volume'),
    'RIS_SP': ('ERIC_startpage',),
    'RIS_EP': ('ERIC_endpage',),
    'RIS_SE': ('ERIC_startpage',),
    'RIS_Y1': ('cf_published_print', 'ERIC_publicationdateyear'),
    'RIS_ET': ('cf_published_online',),
    'RIS_N2': ('cf_abstract',),
    'RIS_UR': ('d_URL_cf_missing', 'cf_URL'),  # Updated here
    # 'RIS_L2': ('cf_link',),  # Removed to prevent duplication
    'RIS_KW': ('cf_subject',),
    'RIS_C1': ('ERIC_subject',),
    'RIS_C2': ('ERIC_description',),
    'RIS_CY': ('cf_publisher_location',),
    'RIS_DB': (),  # Constant value 'ERIC'
    'RIS_C3': ('cf_event',),
    'RIS_N1': ('ERIC_notes',),
    #'RIS_M3': ('cf_type',),
    'RIS_CN': ('cf_event',),
    'RIS_DA': ('ERIC_date',)
}

# Create the 'ris_' columns in the DataFrame
for ris_field, cols in ris_fields.items():
    if ris_field == 'RIS_DB':
        # Assign constant value 'ERIC' for RIS_DB
        phase2d_S1_cf_ris_df[ris_field] = 'ERIC'
    elif ris_field == 'RIS_A1':
        # Handle RIS_A1 specifically
        def get_ris_a1(row):
            cf_author = row.get('cf_author', '')
            eric_author = row.get('ERIC_author', '')
            if cf_author:
                # Authors are separated by semicolons
                authors = [author.strip() for author in cf_author.split(';') if author.strip()]
            elif eric_author:
                # Process ERIC_author
                authors = process_eric_author(eric_author)
            else:
                authors = []
            return '; '.join(authors)
        phase2d_S1_cf_ris_df[ris_field] = phase2d_S1_cf_ris_df.apply(get_ris_a1, axis=1)
    elif ris_field == 'RIS_UR':
        # Implement your specified logic for RIS_UR
        def get_ris_ur(row):
            d_URL_cf_missing = row.get('d_URL_cf_missing', '')
            cf_link = row.get('cf_link', '')
            doi = row.get('d_doi_cf_missing', '') or row.get('cf_DOI', '')
            
            if d_URL_cf_missing == 'No URL':
                return ''
            elif d_URL_cf_missing != 'No URL' and 'doi.org' not in d_URL_cf_missing.lower():
                return d_URL_cf_missing
            elif d_URL_cf_missing != 'No URL' and 'doi.org' in d_URL_cf_missing.lower():
                if cf_link:
                    return cf_link
                elif doi:
                    return 'https://doi.org/' + doi
                else:
                    return ''
            else:
                if doi:
                    return 'https://doi.org/' + doi
                else:
                    return ''
        phase2d_S1_cf_ris_df[ris_field] = phase2d_S1_cf_ris_df.apply(get_ris_ur, axis=1)
    elif ris_field == 'RIS_JO':
        def get_ris_jo(row):
            value = get_ris_value(row, 'cf_journal_name', ['ERIC_source'])
            # Check if there is no URL and if the journal name is in lowercase
            if not row.get('d_URL_cf_missing', '') and value.islower():
                value = value.title()
            return value
        phase2d_S1_cf_ris_df[ris_field] = phase2d_S1_cf_ris_df.apply(get_ris_jo, axis=1)
    elif ris_field == 'RIS_SN':
        def get_ris_sn(row):
            value = get_ris_value(row, 'cf_ISSN', ['ERIC_issn'])
            if value:
                issn_list = [issn.strip() for issn in value.split(',') if issn.strip()]
                return '\n'.join(issn_list)
            return value
        phase2d_S1_cf_ris_df[ris_field] = phase2d_S1_cf_ris_df.apply(get_ris_sn, axis=1)
    else:
        primary_col = cols[0] if len(cols) > 0 else ''
        secondary_cols = cols[1:] if len(cols) > 1 else []
        phase2d_S1_cf_ris_df[ris_field] = phase2d_S1_cf_ris_df.apply(
            lambda row: get_ris_value(row, primary_col, secondary_cols), axis=1)

# ------------------------------
# 4. Export the DataFrame to Excel
# ------------------------------

# Define the output directory and filename
output_dir = r"C:\Users\<USER>\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Phase 2 - Literature Review Search\ERIC API Resources\Search_1_ERIC-CF_Exports"
output_filename = "Search_1_ERIC-CF_RIS_Export.xlsx"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Combine directory and filename to create the full output path
output_excel_path = os.path.join(output_dir, output_filename)

# Write the DataFrame to an Excel file
phase2d_S1_cf_ris_df.to_excel(output_excel_path, index=False)

print(f"Data successfully written to {output_excel_path}")

# ------------------------------
# 5. Optional: Display the First Few Rows
# ------------------------------

# Display the first few rows to verify the RIS columns
display_columns = [col for col in phase2d_S1_cf_ris_df.columns if col.startswith('RIS_')]
display(phase2d_S1_cf_ris_df[display_columns].head())

# ------------------------------
# 6. Convert RIS Columns to RIS File
# ------------------------------

# Define the mapping from RIS columns to RIS tags
# Remove 'RIS_L2' from the mapping to prevent duplication
ris_tag_mapping = {
    'RIS_AN': 'AN',   # Accession Number
    'RIS_TY': 'TY',   # Type of reference
    'RIS_T1': 'TI',   # Primary Title
    'RIS_A1': 'AU',   # Author
    'RIS_DO': 'DO',   # DOI
    'RIS_PB': 'PB',   # Publisher
    'RIS_JO': 'JO',   # Journal
    'RIS_SN': 'SN',   # ISSN
    'RIS_ISBN': 'SN', # ISBN (using SN tag)
    'RIS_IS': 'IS',   # Issue
    'RIS_VL': 'VL',   # Volume
    'RIS_SP': 'SP',   # Start Page
    'RIS_EP': 'EP',   # End Page
    'RIS_SE': 'SE',   # Start Page (non-range)
    'RIS_Y1': 'PY',   # Publication Year
    'RIS_ET': 'ET',   # Edition
    'RIS_N2': 'AB',   # Abstract
    'RIS_UR': 'UR',   # URL
    # 'RIS_L2': 'L2',   # Removed to prevent duplication
    'RIS_KW': 'KW',   # Keywords
    'RIS_C1': 'C1',   # Custom 1
    'RIS_C2': 'C2',   # Custom 2
    'RIS_CY': 'CY',   # Place Published
    'RIS_DB': 'DB',   # Database Name
    'RIS_C3': 'C3',   # Custom 3
    'RIS_N1': 'N1',   # Notes
    #'RIS_M3': 'M3',   # Type of Work
    'RIS_CN': 'CN',   # Call Number
    'RIS_DA': 'DA',   # Date
}

# Create the RIS file content
ris_entries = []

for idx, row in phase2d_S1_cf_ris_df.iterrows():
    ris_entry = []
    # Start with TY - Type of reference
    ty_value = row.get('RIS_TY', 'JOUR')  # Default to 'JOUR' if TY is missing
    ris_entry.append(f"TY  - {ty_value}")
    
    # Handle Authors separately
    authors_value = row.get('RIS_A1', '')
    if authors_value:
        # Authors are separated by semicolons
        authors = [author.strip() for author in authors_value.split(';') if author.strip()]
        for author in authors:
            ris_entry.append(f"AU  - {author}")
    
    # Loop over the ris_tag_mapping
    for ris_col, ris_tag in ris_tag_mapping.items():
        # Skip 'TY' and 'AU' as they are already handled
        if ris_col in ['RIS_TY', 'RIS_A1']:
            continue
        value = row.get(ris_col, '')
        if value:
            if ris_tag == 'KW':
                # Keywords are separated by commas
                keywords = [kw.strip() for kw in value.split(',') if kw.strip()]
                for kw in keywords:
                    ris_entry.append(f"{ris_tag}  - {kw}")
            elif ris_tag == 'SN' and ris_col == 'RIS_SN':
                # ISSNs are separated by line breaks
                issns = value.split('\n')
                for issn in issns:
                    ris_entry.append(f"{ris_tag}  - {issn.strip()}")
            elif ris_tag == 'SN' and ris_col == 'RIS_ISBN':
                # ISBNs are separated by commas
                isbns = [isbn.strip() for isbn in value.split(',') if isbn.strip()]
                for isbn in isbns:
                    ris_entry.append(f"{ris_tag}  - {isbn}")
            else:
                ris_entry.append(f"{ris_tag}  - {value}")
    
    # End of the RIS entry
    ris_entry.append('ER  - ')
    
    # Join the ris_entry list into a string and add to ris_entries
    ris_entries.append('\n'.join(ris_entry))

# Combine all entries into the final RIS content
ris_content = '\n\n'.join(ris_entries)

# Write the RIS content to a file
output_ris_path = os.path.join(output_dir, "Search_1_ERIC-CF_RIS_Export.ris")

with open(output_ris_path, 'w', encoding='utf-8') as file:
    file.write(ris_content)

print(f"RIS file successfully written to {output_ris_path}")


Data successfully written to C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Phase 2 - Literature Review Search\ERIC API Resources\Search_1_ERIC-CF_Exports\Search_1_ERIC-CF_RIS_Export.xlsx


,RIS_AN,RIS_TY,RIS_T1,RIS_A1,RIS_DO,RIS_PB,RIS_JO,RIS_SN,RIS_ISBN,RIS_IS,...,RIS_UR,RIS_KW,RIS_C1,RIS_C2,RIS_CY,RIS_DB,RIS_C3,RIS_N1,RIS_CN,RIS_DA
0,EJ199088,"Journal Articles, Reports - Research",Ability and Group Problem Solving.,"Laughlin, Patrick R.",,,Journal Of Research And Development In Education,,,1,...,,,"Ability Grouping, College Students, Decision Making, Group Structure, Heterogeneous Grouping, High Achievement, Homogeneous Grouping, Low Ability Students, Problem Solving, Teamwork","College students, ranked as high, medium, or low ability, were assigned to homogeneous or mixed-ability groups of two through five members for a test. Performance proved proportional to the number of high ability group members. The underlying group process was inferred through social decision scheme analysis. (Author/SJL)",,ERIC,,,,Fall
1,EJ198017,journal-article,Mathematics and Learning Disabled Youth: The Upper Grade Levels,"Cawley, John F.; Fitzmaurice, Anne M.; Shaw, Robert A.; Kahn, Harris; Bates, Herman",10.2307/1510976,SAGE Publications,Learning Disability Quarterly,0731-9487\n2168-376X,,4,...,https://journals.sagepub.com/doi/pdf/10.2307/1510976,,"Concept Formation, Educational Assessment, Fractions, Geometry, Individualized Instruction, Learning Disabilities, Mathematics Instruction, Measurement, Numbers, Secondary Education, Skill Development, Student Placement",The article presents a revised model for individualized mathematics programing for the learning disabled adolescent. (PHR),,ERIC,,,,Fall
2,EJ198868,journal-article,The Comparative Validity of the California State University and Colleges English Placement Test (CSUC-EPT) in the Prediction of Fall Semester Grade Point Average and English Course Grades of First-Semester Entering Freshmen,"Michael, William B.; Shaffer, Phyllis",10.1177/001316447803800418,SAGE Publications,Educational and Psychological Measurement,0013-1644\n1552-3888,,4,...,https://journals.sagepub.com/doi/pdf/10.1177/001316447803800418,,"College Freshmen, English Education, Equivalency Tests, Higher Education, Predictive Validity, Predictor Variables, Reading Tests, Student Placement, Technical Reports, Test Reliability, Writing Skills",The California State University and Colleges English Placement Test was designed to provide information about the reading and writing skills of newly admitted college students. The validity of this measure is examined. (Author/JKS),,ERIC,,,,Win
3,EJ207275,"Journal Articles, Reports - Research",Instructor Assessment of a Standardized Proficiency Test for Course Exemption.,"Hargrett, Nancy T.; Chapman, David W.",,,Educational Research Quarterly,,,3,...,,,"Advanced Placement, College Credits, Equivalency Tests, Higher Education, Predictive Measurement, Student Placement, Teacher Attitudes, Technical Reports, Test Reviews",The use of standardized test scores to exempt students from college courses was investigated using the General Psychology Exam of the College Level Examination Program. Results questioned the ability of instructors to assess its utility. (JKS),,ERIC,,,,Fall
4,EJ221720,journal-article,Academic Self-Concept Change in Special Education Students: Some Suggestions for Interpreting Self-Concept Scores',"Boersma, Frederic J.; Chapman, James W.; Battle, James",10.1177/002246697901300410,SAGE Publications,The Journal of Special Education,0022-4669\n1538-4764,,4,...,https://journals.sagepub.com/doi/pdf/10.1177/002246697901300410,,"Academic Achievement, Disabilities, Elementary Education, Exceptional Child Research, Learning Disabilities, Mainstreaming, Mild Mental Retardation, Self Concept, Student Placement","Changes in academic self-concept were measured by Student's Perception of Ability Scale in 50 learning-disabled students, 18 educable mentally retarded students receiving full-time remedial placement, and 83 regular-class students. Pre-post data collected over 12-month period revealed that full-time placement was accompanied by statistically significant increase

RIS file successfully written to C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Phase 2 - Literature Review Search\ERIC API Resources\Search_1_ERIC-CF_Exports\Search_1_ERIC-CF_RIS_Export.ris


## Find & Download PDFS

### Automated PDF Download from Sci-Hub using DOIs

This notebook automates the process of downloading PDFs using DOIs from a dataset by querying various Sci-Hub mirrors. It organizes the downloaded files into decade-specific folders with appropriate naming conventions.
Steps in the Workflow:

<b>1. Prepare the Environment:</b>
<ul> <li>Define the base folder location for storing PDFs.</li> <li>Set up a rotating list of user-agent strings to mimic different browsers for better access.</li> </ul>

<b>2. Define Helper Functions:</b>
<ul> <li><code>sanitize_filename()</code>: Removes invalid characters from filenames.</li> <li><code>shorten_title()</code>: Shortens the title by concatenating and capitalizing a few words.</li> <li><code>create_filename()</code>: Constructs the filename based on author, year, and title.</li> <li><code>get_decade_folder()</code>: Determines the correct decade folder for organizing PDFs.</li> <li>Custom functions for extracting PDF links from HTML content for different Sci-Hub mirrors.</li> </ul>

<b>3. Main PDF Download Function:</b>
<ul> <li><code>download_pdf()</code>: Downloads the PDF based on the DOI and saves it to the appropriate decade folder.</li> <li>Handles retries and rotates through different user-agents to avoid request blocks.</li> <li>Attempts multiple Sci-Hub mirrors until a successful download is completed or retries are exhausted.</li> </ul>

<b>4. Execution Loop:</b>
<ul> <li>Iterates through the dataset and calls <code>download_pdf()</code> for each entry.</li> </ul>

<b>Output</b>:
<ul> <li><b>PDF Files:</b> The notebook saves PDFs in decade-specific subfolders within the base folder. The filenames follow the format: <ul> <li>Single author: <code>rsch_YEAR_LastName_ShortTitle.pdf</code></li> <li>Two authors: <code>rsch_YEAR_LastName1.LastName2_ShortTitle.pdf</code></li> <li>Three or more authors: <code>rsch_YEAR_LastName et al_ShortTitle.pdf</code></li> </ul> </li> <li><b>Logs:</b> The notebook prints status updates and error messages to indicate progress and any issues encountered during the download process.</li> </ul>

<b>Prerequisites</b>:
<ul> <li><b>Python Libraries:</b> <ul> <li><code>pandas</code>: DataFrame handling</li> <li><code>requests</code>: Making HTTP requests</li> <li><code>BeautifulSoup4</code>: HTML parsing for extracting PDF links</li> <li><code>re</code>: Regular expressions for text processing</li> <li><code>os</code>: File path handling</li> <li><code>time</code>: Pausing between retries</li> </ul> </li> <li><b>Environment Setup:</b> <ul> <li>Ensure the Python environment has all required libraries installed.</li> <li>Access to Sci-Hub mirrors that may be restricted in some locations.</li> </ul> </li> </ul>

In [32]:
import pandas as pd
import requests
from tqdm import tqdm  # Standard tqdm, not notebook version

# Assuming 'phase2d_S1_cf_ris_df' is your DataFrame containing DOIs
df_pdf = phase2d_S1_cf_ris_df.copy()

# Create new columns for PDF availability and link
df_pdf['PDF_Available'] = ''
df_pdf['PDF_Link'] = ''

# List of Sci-Hub mirrors to try
sci_hub_mirrors = [
    'https://sci-hub.wf/',
    'https://sci-hub.st/',
    'https://sci-hub.ee/'
]

# Function to check for PDF availability using mirrors
def fetch_pdf_url(doi):
    for mirror in sci_hub_mirrors:
        url = f"{mirror}{doi}"
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            if response.status_code == 200:
                return 'Yes', url  # Assuming the mirror redirects to the PDF or a download link
            else:
                continue
        except requests.HTTPError:
            continue  # Try the next mirror if HTTP error occurs
    return 'Error', 'All mirrors failed'

# Enable tqdm for pandas with standard tqdm
tqdm.pandas(desc="Searching PDFs", leave=False)

# Apply function to each row using progress_apply
def check_pdf_availability(row):
    doi = row['RIS_DO']
    if pd.isna(doi) or doi == '':
        return 'No', ''
    else:
        return fetch_pdf_url(doi)

# Update DataFrame using progress_apply
df_pdf[['PDF_Available', 'PDF_Link']] = df_pdf.progress_apply(
    lambda row: check_pdf_availability(row), axis=1, result_type='expand'
)

# Display the DataFrame for all articles
display(df_pdf[['RIS_T1', 'RIS_Y1', 'RIS_DO', 'PDF_Available', 'PDF_Link']])


NameError: name 'phase2d_S1_cf_ris_df' is not defined

## Download the PDFs

This section of the Jupyter Notebook automates the download of PDF files from various Sci-Hub mirrors using DOIs provided in a dataset. The downloaded files are organized into decade-specific subfolders, ensuring they are properly named and stored for easy access and analysis.
Steps in the Workflow:

<b>1. Environment Setup:</b>
<ul> <li><b>Base Folder:</b> Define the base directory for saving downloaded PDFs. Replace the placeholder `<USER>` with the specific username or use environment variables to avoid exposing sensitive paths.</li> <li><b>User-Agent Rotation:</b> A list of user-agents is rotated to mimic different browsers, enhancing the ability to bypass website restrictions.</li> </ul>

<b>2. PDF Download Helper Functions:</b>
<ul> <li><code>sanitize_filename()</code>: Cleans filenames by removing invalid characters.</li> <li><code>shorten_title()</code>: Shortens the title to a specified number of words, capitalizing and concatenating them.</li> <li><code>create_filename()</code>: Creates the filename based on author, publication year, and shortened title.</li> <li><code>get_decade_folder()</code>: Determines the appropriate decade folder based on the publication year.</li> <li><b>Custom Extraction Functions:</b> Functions for extracting PDF links from the HTML content of different Sci-Hub mirrors, tailored to their unique structures.</li> </ul>

<b>3. Main PDF Download Function:</b>
<ul> <li><code>download_pdf()</code>: The primary function that: <ul> <li>Checks if the PDF is available and extracts the DOI link for download.</li> <li>Iterates over multiple Sci-Hub mirrors and retries connections in case of failures.</li> <li>Uses user-agent rotation to avoid being blocked by servers.</li> <li>Handles different content types and extracts direct PDF links using BeautifulSoup.</li> </ul> </li> <li>Ensures that downloaded PDFs are stored in decade-specific folders and are named following a structured convention.</li> </ul>

<b>4. Execution Loop:</b>
<ul> <li>Iterates over the rows of the DataFrame <code>df_pdf</code> and calls <code>download_pdf()</code> for each entry.</li> <li>Includes error handling and retry logic for connection issues.</li> </ul>
<b>Output:</b>
<ul> <li><b>PDF Files:</b> The downloaded PDFs are stored in organized subfolders named by decade (e.g., <code>1970-1979</code>, <code>1980-1989</code>).</li> <li><b>File Naming Convention:</b> <ul> <li>Single author: <code>rsch_YEAR_LastName_ShortTitle.pdf</code></li> <li>Two authors: <code>rsch_YEAR_LastName1.LastName2_ShortTitle.pdf</code></li> <li>Three or more authors: <code>rsch_YEAR_LastName et al_ShortTitle.pdf</code></li> </ul> </li> <li><b>Logs:</b> Status updates and error messages are printed to indicate progress and any issues encountered during the download process.</li> </ul>
<b>Prerequisites:</b>
<ul> <li><b>Python Libraries:</b> <ul> <li><code>pandas</code>: DataFrame handling</li> <li><code>requests</code>: HTTP requests for downloading PDFs</li> <li><code>BeautifulSoup4</code>: Parsing HTML content to extract PDF links</li> <li><code>re</code>: Regular expressions for text processing</li> <li><code>os</code>: File path management</li> <li><code>time</code>: Adding delays between retry attempts</li> </ul> </li> <li><b>Environment Setup:</b> Ensure a Python environment with the above libraries installed and access to Sci-Hub mirrors (note that access to Sci-Hub may be restricted in some regions).</li> </ul>

In [100]:
import os
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

# Base folder location
base_folder = r"C:\Users\<USER>\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Literature (PDFs)\Phase 2\Search_1_Articles"

# Rotating user-agent list to mimic different browsers
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:45.0) Gecko/20100101 Firefox/45.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15'
]

# Sci-Hub mirrors to try
sci_hub_mirrors = [
    'https://sci-hub.wf/',
    'https://sci-hub.st/',
    'https://sci-hub.se/',
    'https://sci-hub.ru/'
]

# Function to sanitize file names by removing invalid characters
def sanitize_filename(filename):
    return re.sub(r'[\/:*?"<>|]', '', filename)

# Function to shorten the title (concatenate words and capitalize)
def shorten_title(title, max_words=3):
    words = title.split()[:max_words]  # Take the first max_words
    return ''.join(word.capitalize() for word in words)  # Capitalize each word and concatenate

# Function to create the naming structure
def create_filename(authors, year, title):
    short_title = shorten_title(title)
    
    if len(authors) == 1:
        filename = f"rsch_{year}_{authors[0].split()[0]}_{short_title}.pdf"
    elif len(authors) == 2:
        filename = f"rsch_{year}_{authors[0].split()[0]}.{authors[1].split()[0]}_{short_title}.pdf"
    else:
        filename = f"rsch_{year}_{authors[0].split()[0]} et al_{short_title}.pdf"
    
    return sanitize_filename(filename)

# Function to get the decade folder
def get_decade_folder(year):
    if 1970 <= year <= 1979:
        return "1970-1979"
    elif 1980 <= year <= 1989:
        return "1980-1989"
    elif 1990 <= year <= 1999:
        return "1990-1999"
    elif 2000 <= year <= 2009:
        return "2000-2009"
    elif 2010 <= year <= 2019:
        return "2010-2019"
    else:
        return "2020-Present"

# Custom function to extract the PDF link from sci-hub.wf
def extract_pdf_link_wf(html_content, doi):
    soup = BeautifulSoup(html_content, 'html.parser')
    iframe = soup.find('iframe', {'id': 'pdf'})
    if iframe:
        return iframe['src']
    # Fallback: construct the link using the sci.bban.top pattern
    return f'https://sci.bban.top/pdf/{doi}.pdf'

def extract_pdf_link_st(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    embed = soup.find('embed', {'id': 'pdf'})
    if embed:
        return 'https:' + embed['src'] if embed['src'].startswith('//') else embed['src']
    return None

def extract_pdf_link_se(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    button = soup.find('button', text='save')
    if button and 'location.href' in button['onclick']:
        return 'https:' + button['onclick'].split("'")[1] if button['onclick'].startswith('//') else button['onclick'].split("'")[1]
    return None

def extract_pdf_link_ru(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    embed = soup.find('embed', {'id': 'pdf'})
    if embed:
        return 'https:' + embed['src'] if embed['src'].startswith('//') else embed['src']
    return None

# Main function to download the PDF and save it to the appropriate folder
def download_pdf(row, retries=3):
    if row['PDF_Available'] == 'Yes' and row['PDF_Link']:
        year = row['ERIC_publicationdateyear']
        authors = row['ERIC_author'].split(", ")  # Splitting authors based on comma in your dataset
        title = row['ERIC_title']
        
        # Get the decade folder
        decade_folder = get_decade_folder(year)
        save_path = os.path.join(base_folder, decade_folder)
        
        # Ensure the decade folder exists
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # Create the file name
        filename = create_filename(authors, year, title)
        file_path = os.path.join(save_path, filename)
        
        # Try multiple Sci-Hub mirrors with custom extraction functions
        for mirror in sci_hub_mirrors:
            attempt = 0
            pdf_url = f"{mirror}{row['RIS_DO']}"  # Generate the URL based on DOI and mirror
            
            # Choose the correct extraction function based on the mirror
            if 'sci-hub.wf' in mirror:
                extract_pdf_func = lambda html: extract_pdf_link_wf(html, row['RIS_DO'])
            elif 'sci-hub.st' in mirror:
                extract_pdf_func = extract_pdf_link_st
            elif 'sci-hub.se' in mirror:
                extract_pdf_func = extract_pdf_link_se
            elif 'sci-hub.ru' in mirror:
                extract_pdf_func = extract_pdf_link_ru
            else:
                continue  # Unknown mirror, skip it
            
            while attempt < retries:
                try:
                    # Rotate through different user-agents
                    headers = {'User-Agent': user_agents[attempt % len(user_agents)]}
                    
                    response = requests.get(pdf_url, headers=headers)
                    
                    # Check if the content type is HTML and not a direct PDF
                    if response.headers['Content-Type'].startswith('text/html'):
                        # Parse the HTML and extract the PDF link using the appropriate function
                        pdf_iframe_url = extract_pdf_func(response.content)
                        if pdf_iframe_url:
                            # Now download the actual PDF
                            pdf_response = requests.get(pdf_iframe_url, headers=headers, stream=True)
                            if pdf_response.headers['Content-Type'] == 'application/pdf':
                                with open(file_path, 'wb') as f:
                                    f.write(pdf_response.content)
                                print(f"Downloaded: {file_path}")
                                return  # Success, exit the function
                            else:
                                print(f"Failed to download PDF from iframe: {pdf_iframe_url}")
                        else:
                            print(f"No PDF link found in: {pdf_url}")
                    else:
                        print(f"Content-Type is not HTML: {response.headers['Content-Type']}")
                        break  # Stop retrying if it's not HTML
                
                except requests.exceptions.ConnectionError as e:
                    print(f"Error downloading {pdf_url}: {e}")
                    time.sleep(5)  # Wait for 5 seconds before retrying after a connection error
                
                except Exception as e:
                    print(f"Unexpected error downloading {pdf_url}: {e}")
                
                attempt += 1
                time.sleep(2)  # Wait for 2 seconds before retrying
            
            print(f"Failed to download from {mirror} after {retries} attempts: {pdf_url}")

# Loop through the DataFrame df_pdf and download PDFs
for idx, row in df_pdf.iterrows():  # Testing with the first 10 rows
    download_pdf(row)

Downloaded: C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Literature (PDFs)\Phase 2\Search_1_Articles\1970-1979\rsch_1978_Cawley et al_MathematicsAndLearning.pdf
Downloaded: C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Literature (PDFs)\Phase 2\Search_1_Articles\1970-1979\rsch_1978_Michael et al_TheComparativeValidity.pdf
Downloaded: C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Literature (PDFs)\Phase 2\Search_1_Articles\1970-1979\rsch_1979_Boersma et al_AcademicSelf-conceptChange.pdf
Failed to download PDF from iframe: https://sci.bban.top/pdf/10.1016/0022-4405(79)90017-7.pdf
Failed to download PDF from iframe: https://sci.bban.top/pdf/10.1016/0022-4405(79)90017-7.pdf
Failed to download PDF from iframe: https://sci.bban.top/pdf/10.1016/0022-4405(79)90017-7.pdf
Failed to download from https://sci-hub.wf/ af

C:\Users\sbaser\AppData\Local\Temp\ipykernel_4028\1086431848.py:81: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  button = soup.find('button', text='save')


No PDF link found in: https://sci-hub.se/10.1080/00220671.1980.10885250
No PDF link found in: https://sci-hub.se/10.1080/00220671.1980.10885250
No PDF link found in: https://sci-hub.se/10.1080/00220671.1980.10885250
Failed to download from https://sci-hub.se/ after 3 attempts: https://sci-hub.se/10.1080/00220671.1980.10885250
No PDF link found in: https://sci-hub.ru/10.1080/00220671.1980.10885250
No PDF link found in: https://sci-hub.ru/10.1080/00220671.1980.10885250
No PDF link found in: https://sci-hub.ru/10.1080/00220671.1980.10885250
Failed to download from https://sci-hub.ru/ after 3 attempts: https://sci-hub.ru/10.1080/00220671.1980.10885250
Downloaded: C:\Users\sbaser\OneDrive - University of Georgia\Shared Folders\P-20 Parallels and Perils\Data Collection\Literature (PDFs)\Phase 2\Search_1_Articles\1980-1989\rsch_1980_Smart et al_TeacherFactorsAnd.pdf
Failed to download PDF from iframe: https://sci.bban.top/pdf/10.1002/1520-6807(198010)17:4<446::aid-pits2310170405>3.0.co;2-7.pd